### -> Run all cells to update the news dataset

In [22]:
import pandas as pd

In [23]:
# news = pd.read_csv("/workspaces/thesis/assets/nefino_news_until_feb_08_2024.csv")
news = pd.read_csv("/workspaces/thesis/assets/nefino_news_until_march_30_2024.csv")

/tmp/ipykernel_90881/2714900528.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  news = pd.read_csv("/workspaces/thesis/assets/nefino_news_until_march_30_2024.csv")


In [25]:
# get the latest date
latest_date = news["date"].max()
latest_date

'2024-03-28'

In [26]:
count_summary_but_no_headline = len(
    news[(news["headline"] == "empty_headline") & (news["summary"].notnull())]
)
count_summary_and_headline = len(
    news[
        (~news["headline"].isin(["empty_headline", ""]))
        & (news["headline"].notnull())
        & (news["summary"].notnull())
    ]
)

print("Summary but no headline:", count_summary_but_no_headline)
print("Summary and headline:", count_summary_and_headline)

# Keep only news with summary
news = news[news["summary"].notnull() & (news["summary"] != "")]

Summary but no headline: 2102
Summary and headline: 18614


In [27]:
# drop duplicates and null values of summary and headline
news = news.drop(
    news[
        (news["duplicate"] == True)
        | (news["summary"].isnull() | news["headline"].isnull())
    ].index
)

# drop unnecessary columns
news = news.drop(
    columns=[
        # secret Nefino columns
    ]
)

In [28]:
# merge columns 3 to 6 into a single column called place_id
news = news.assign(
    place_id=news["local_administrative_unit_id"].fillna("")
    + news["country_id"].fillna("")
    + news["federal_state_id"].fillna("")
    + news["planning_region_id"].fillna("")
)

# drop merged columns
news = news.drop(
    columns=[
        "local_administrative_unit_id",
        "country_id",
        "federal_state_id",
        "planning_region_id",
    ]
)

In [29]:
news["relevancy_by_ai"].value_counts()

relevancy_by_ai
1    10723
3      934
2      627
4      295
Name: count, dtype: int64

In [30]:
# drop rows where ai relevancy is not 1 whil1e the editor relevancy is null
news = news.drop(
    news[
        ((news["relevancy_by_ai"] != 1) & (news["relevancy_by_editor"].isnull()))
        | (news["relevancy_by_editor"] == 2)
    ].index
)
news = news.drop(columns=["relevancy_by_editor", "relevancy_by_ai"])

In [31]:
# drop all rows except one if summary field is the same
news = news.drop_duplicates(subset="summary", keep="first")

In [32]:
# count the number of place_id which are an emtpy string
news["place_id"].value_counts().loc[""]

2243

In [33]:
# Sort the news dataframe by place_id
news = news.sort_values(by='place_id')

# Update the page_ptr_id column
news['page_ptr_id'] = range(1, len(news) + 1)
news = news.rename(columns={'page_ptr_id': 'news_id'})


In [34]:
news[news["energy_types"].str.contains("SO")].count()

news["energy_types"] = (
    news["energy_types"].str.replace("SO", "SOLAR").replace("WI", "WIND")
)

# create solar and solar/wind datasets
news_solar = news[news["energy_types"].str.contains("SO")]
news_solar_wind = news.copy()

In [35]:
news_solar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4405 entries, 35231 to 40012
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   news_id       4405 non-null   int64 
 1   date          4405 non-null   object
 2   news_type     4405 non-null   int64 
 3   energy_types  4405 non-null   object
 4   summary       4405 non-null   object
 5   headline      4405 non-null   object
 6   place_id      4405 non-null   object
dtypes: int64(2), object(5)
memory usage: 275.3+ KB


In [36]:
news_solar_wind.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10571 entries, 35231 to 40012
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   news_id       10571 non-null  int64 
 1   date          10571 non-null  object
 2   news_type     10571 non-null  int64 
 3   energy_types  10571 non-null  object
 4   summary       10571 non-null  object
 5   headline      10571 non-null  object
 6   place_id      10571 non-null  object
dtypes: int64(2), object(5)
memory usage: 660.7+ KB


In [37]:
news_solar_wind.head(2)

,news_id,date,news_type,energy_types,summary,headline,place_id
35231,1,2023-12-22,3,{SOLAR},Der Energiekonzern Vattenfall plant den Bau vo...,Stadt Nauen und Gemeinde Silberstedt: Vattenfa...,
34990,2,2023-12-20,2,{WI},Die Landesregierung von Schleswig-Holstein hat...,Schleswig-Holstein: Landesregierung plant Erhö...,


In [38]:
# news[news['place_id'] == ''].count()

In [39]:
# export to csv
news_solar.to_csv(
    '/workspaces/thesis/assets/nefino_solar_news_until_{}_cleaned.csv'.format(latest_date.replace('-', '_')),
    index=False,
)

news_solar_wind.to_csv(
    '/workspaces/thesis/assets/nefino_solar_and_wind_news_until_{}_cleaned.csv'.format(latest_date.replace('-', '_')),
    index=False,
)

In [41]:
# count number of rows with no place_id
news_solar[news_solar["place_id"] == ""].count()

news_solar_wind[news_solar_wind["place_id"] == ""].count()

news_id         2243
date            2243
news_type       2243
energy_types    2243
summary         2243
headline        2243
place_id        2243
dtype: int64